# Milieugevolgen van de Uitstoot van Broeikasgassen
![image](images/home8.png)

In [ ]:
import plotly.graph_objs as go
import plotly.express as px
import pandas as pd
import numpy as np
from scipy.stats import pearsonr

nd_df = pd.read_csv('https://raw.githubusercontent.com/Elijah-1/IV-dataset-emissions/main/disaster-events%20new.csv')
ghg_df = pd.read_csv('https://raw.githubusercontent.com/Elijah-1/IV-dataset-emissions/main/transformed_data_separate_columns.csv')

# Load the emissions data
#emissions_data_country = pd.read_csv('https://raw.githubusercontent.com/Elijah-1/IV-dataset-emissions/main/global-co2-fossil-plus-land-use.csv')

# remove last year and 1970- years from natural disaster dataset
nd_df = nd_df[nd_df['Year'] >= 1970]
nd_df = nd_df[nd_df['Year'] != 2023]
nd_df = nd_df[nd_df['Year'] != 2024]

# emissions only all per year
ghg_all_df = ghg_df[ghg_df['Substance'] == 'ALL'][['Year', 'Emissions']]

# emissions excluding all per year
ghg_ex_df = ghg_df[ghg_df['Substance'] != 'ALL']

# natural disasters only all disasters per year
nd_all_df = nd_df[nd_df['Entity'] == 'All disasters'][['Year', 'Disasters']]

# natural disasters excluding all disasters {wildcard} per year
nd_ex_df = nd_df[~nd_df['Entity'].str.startswith('All disasters')]

# Extract disaster data for "all disasters"
nd_all_disasters_df = nd_df[nd_df['Entity'] == 'All disasters']

# merged
merged_all_df = pd.merge(ghg_all_df, nd_all_df, on='Year', how='inner')

disaster_counts = nd_ex_df.groupby('Entity')['Disasters'].sum().reset_index()

De wereld wordt steeds meer bedreigd door natuurrampen, een probleem dat verergerd wordt door de groeiende uitstoot van broeikasgassen. Mensen die in kwetsbare gebieden wonen, zijn ervan overtuigd dat de toename in frequentie en hevigheid van deze rampen direct verband houdt met de stijgende emissies. Dit heeft verwoestende effecten op hun levens, eigendommen en lokale economieën. Studies tonen aan dat broeikasgassen, veroorzaakt door menselijke activiteiten, de klimaatverandering versnellen. Dit leidt tot extremere weersomstandigheden zoals overstromingen en hittegolven. Anderen beweren echter dat veel van deze veranderingen te wijten zijn aan natuurlijke variaties in het klimaat en dat de impact van broeikasgassen wordt overdreven en dat adaptieve strategieën en technologieën voldoende zijn om eventuele negatieve effecten te mitigeren. Deze uiteenlopende perspectieven suggereren dat er verschillen in interpretatie en prioriteit zijn bij het aanpakken van de gevolgen van klimaatverandering.

Als belangrijkste databronnen gebruiken we de EDGAR-dataset, die ons gedetailleerde informatie verstrekt over de uitstoot van broeikasgassen van 1970-2022. Daarnaast maken we gebruik van een dataset die gegevens bevat over het aantal en de soorten natuurrampen die hebben plaatsgevonden van 1900-2024. Om de correlatie tussen natuurrampen en de uitstoot van broeikasgassen te begrijpen en te onderzoeken, zullen we factoren zoals emissies, natuurrampen en de Pearson-correlatiecoëfficiënt tussen de twee analyseren. Verder zullen we relevante literatuur raadplegen om de verschillende argumenten te onderbouwen.


### Natuurlijke rampen

In [ ]:
disaster_types = nd_ex_df['Entity'].unique()
colors = px.colors.qualitative.T10[:len(disaster_types)]
color_map = {disaster: colors[i % len(colors)] for i, disaster in enumerate(disaster_types)}

# Create the stacked bar plot
fig_bar = go.Figure()

for disaster in disaster_types:
    disaster_data = nd_ex_df[nd_ex_df['Entity'] == disaster]
    fig_bar.add_trace(go.Bar(
        x=disaster_data['Year'],
        y=disaster_data['Disasters'],
        name=disaster,
        marker=dict(color=color_map[disaster])
    ))

fig_bar.update_layout(
    barmode='stack',
    title='Jaarlijkse natuurramp frequentie per type',
    xaxis=dict(title='Jaar', titlefont=dict(size=14), tickfont=dict(size=12)),
    yaxis=dict(title='Aantal rampen', titlefont=dict(size=14), tickfont=dict(size=12)),
    legend=dict(
        title='Ramptype',
        font=dict(size=12),
        orientation='h',
        yanchor='top',
        y=-0.15,  # Adjust this value to move the legend further below the graph
        xanchor='center',
        x=0.5
    ),
    height=600  # Increase height for better visibility
)

fig_bar.show()

> Figuur 1: Deze grafiek toont het aantal natuurrampen van het jaar 1970-2022, onderverdeeld naar type ramp met behulp van kleuren. Beweeg de muis erover om het precieze aantal rampen te zien. 

Deze grafiek laat het aantal natuurrampen zien van 1970 tot 2022, onderverdeeld in verschillende categorieën. Er is een duidelijke toename te zien van het aantal natuurrampen tussen 1970 en 2000. Na 2000 is er echter geen significante stijging of daling meer waarneembaar, wat resulteert in een vergelijkbaar aantal natuurrampen in 2022 ten opzichte van 2000.

In [ ]:
fig_pie = go.Figure(go.Pie(
    labels=disaster_counts['Entity'],
    values=disaster_counts['Disasters'],
    hole=0.8,
    marker=dict(colors=[color_map[disaster] for disaster in disaster_counts['Entity']]),
    textposition='outside',
    textinfo='percent'
))

fig_pie.update_layout(
    title_text='Frequentie van verschillende rampen',
)

fig_pie.show()

> Figuur 2: De grafiek laat de distributie zien van alle natuurrampen door hun percentages te geven. Beweeg de muis over <br> de sectie om het precieze aantal te zien. De kleuren van de natuurrampen komen overeen met de vorige grafiek.

Grafiek 2 is een cirkeldiagram dat de verdeling van typen natuurrampen laat zien van 1970 tot 2022. Uit de grafiek blijkt dat overstromingen de meest voorkomende natuurramp zijn, gevolgd door extreme weersomstandigheden. Deze twee typen natuurrampen kunnen worden veroorzaakt door de stijgende broeikasgasuitstoot, die leidt tot hogere temperaturen.

### Groei van emissies

Een belangrijke factor om te overwegen is de groei van de broeikasgasuitstoot. In 1970 bedroeg de totale broeikasgasuitstoot ongeveer 27 gigaton. Hoewel er af en toe dalingen waren, is de uitstoot over het algemeen aanzienlijk gestegen van 1970 tot 2022. In 2022 bedroeg de totale broeikasgasuitstoot ongeveer 53 gigaton, wat meer dan een verdubbeling ten opzichte van 1970.

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=ghg_all_df['Year'], y=ghg_all_df['Emissions'], mode='lines+markers', name='Emissions'))
fig.update_layout(
    title='Mondiale uitstoot van broeikasgassen door de jaren heen',
    xaxis_title='Jaar',
    yaxis_title='Emissies',
    template='plotly_white',
   # paper_bgcolor='#d1e8d2',  # Background color for the whole output
   # plot_bgcolor='#d1e8d2'
)

fig.show()

> Figuur 3: Deze grafiek laat de trends zien in broeikasgasemissies van 1970 tot 2022. Over het algemeen vertoont de grafiek een toename van de broeikasgasuitstoot. Met name valt op dat de stijging tussen 2000 en 2022 aanzienlijk sterker is dan tussen 1970 en 2000.

Ondanks de enorme stijging in broeikasgasuitstoot, toont grafiek 1 aan dat de groei van het aantal natuurrampen beperkt is gebleven. De grafiek laat zien dat er na 2000 weliswaar een algemene stijging is in het aantal natuurrampen, maar deze stijging is niet in verhouding met de toename van de broeikasgasuitstoot. Dit suggereert dat het aantal natuurrampen niet sterk is beïnvloed door de groei van de broeikasgasuitstoot.

### De mogelijke gevolgen van emissieproductie door de sectoren


In [ ]:
aggregated_df = ghg_ex_df.groupby(['Year', 'Sector'], as_index=False)['Emissions'].sum()

# Pivot the dataframe to have years as rows and sectors as columns
pivot_df = aggregated_df.pivot(index='Year', columns='Sector', values='Emissions')

# Create the figure
fig = go.Figure()

# Add traces for each sector
for sector in pivot_df.columns:
    fig.add_trace(go.Scatter(
        x=pivot_df.index,
        y=pivot_df[sector],
        mode='lines+markers',
        name=sector,
    ))

# Update layout
fig.update_layout(
    title="Broeikasgasemissies per sector",
    xaxis_title="Jaar",
    yaxis_title="Emissies",
    legend_title="Sector",
    hovermode="x",
    template="plotly_white",
    legend=dict(
        orientation="h",
        yanchor="top",
        y=-0.2,  # Adjust this value to move the legend further down
        xanchor="center",
        x=0.5
    )
)

# Show the figure
fig.show()

> Figuur 4: De grafiek toont de uitstoot van broeikasgassen (GHG) per sector van 1970 tot 2022, met sectoren zoals landbouw, gebouwen, brandstofwinning, industriële verbranding, de energiesector, processen, transport en afval.

Duidelijke trends zijn zichtbaar: de energiesector vertoont een significante toename in uitstoot, terwijl sectoren als afval en transport relatief stabiel zijn gebleven.
De grafiek biedt inzicht in de historische ontwikkelingen van GHG-uitstoot per sector. Deze informatie is cruciaal voor het ontwikkelen van strategieën tegen klimaatverandering en helpt beleidsmakers en onderzoekers om beter geïnformeerde beslissingen te nemen.

In [ ]:
sectors = ghg_ex_df['Sector'].unique()
disaster_types = nd_ex_df['Entity'].unique()

def calculate_pearson(df1, df2, sector):
    correlations = []
    for disaster in disaster_types:
        merged_df = pd.merge(
            df1[(df1['Sector'] == sector)][['Year', 'Emissions']],
            df2[(df2['Entity'] == disaster)][['Year', 'Disasters']],
            on='Year'
        )
        if not merged_df.empty:
            corr, _ = pearsonr(merged_df['Emissions'], merged_df['Disasters'])
            correlations.append(corr)
        else:
            correlations.append(None)
    return correlations

def calculate_all_disasters_pearson(df1, df2, sector):
    merged_df = pd.merge(
        df1[(df1['Sector'] == sector)][['Year', 'Emissions']],
        df2[['Year', 'Disasters']],
        on='Year'
    )
    if not merged_df.empty:
        corr, _ = pearsonr(merged_df['Emissions'], merged_df['Disasters'])
        return corr
    else:
        return None

# Initial calculation for the first sector
initial_sector = sectors[0]
correlations = calculate_pearson(ghg_ex_df, nd_ex_df, initial_sector)
all_disasters_correlation = calculate_all_disasters_pearson(ghg_ex_df, nd_all_disasters_df, initial_sector)

# Define colors
colors = px.colors.qualitative.T10

# Create the initial bar plot
fig = go.Figure(data=[
    go.Bar(name='Individual Disasters', x=disaster_types, y=correlations, marker_color=colors),
    go.Bar(name='All Disasters', x=['All Disasters'], y=[all_disasters_correlation], marker_color=colors[1])
])

fig.update_layout(
    title=f'Pearson correlatie over diverse sectoren',
    xaxis_title='Ramptype',
    yaxis_title='Pearson correlatiecoëfficiënt',
    barmode='group',
    showlegend=False  # Remove the legend
)

# Create the dropdown menu
dropdown_buttons = [
    dict(label=sector,
         method='update',
         args=[{
             'y': [calculate_pearson(ghg_ex_df, nd_ex_df, sector),
                   [calculate_all_disasters_pearson(ghg_ex_df, nd_all_disasters_df, sector)]],
             'marker': [
                 {'color': colors[:len(disaster_types)]},  # Colors for individual disasters
                 {'color': [colors[1]]}  # Color for all disasters
             ],
             'title': f'Pearson Correlation for {sector} Sector'
         }])
    for sector in sectors
]

fig.update_layout(
    updatemenus=[
        dict(
            buttons=dropdown_buttons,
            direction='down',
            pad={'r': 10, 't': 10},
            showactive=True,
            x=0.1,
            xanchor='left',
            y=1.1,
            yanchor='top'
        )
    ]
)

fig.show()

> Figuur 5: De grafieken tonen de Pearson-correlatiecoëfficiënten voor verschillende rampen en hun impact op diverse sectoren. Met de knop kun je selecteren welke sector je wilt bekijken.

Extreem weer en overstromingen hebben consequent de hoogste positieve correlaties in meerdere sectoren, waaronder landbouw, brandstofwinning, industriële verbranding, de energiesector, processen, transport en afval. Dit wijst erop dat deze rampen de grootste invloed hebben op deze sectoren. In de landbouwsector vertonen extreem weer en overstromingen de grootste impact, terwijl droogte een gematigde positieve correlatie heeft. Bij gebouwen zijn aardbevingen en extreem weer het meest invloedrijk. Over het geheel genomen benadrukken de grafieken dat extreem weer en overstromingen de grootste bedreigingen vormen voor diverse economische sectoren, wat cruciale informatie biedt voor beleidsmakers en onderzoekers bij het ontwikkelen van veerkracht strategieën.

### Is er een relatie?

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=merged_all_df['Emissions'], 
    y=merged_all_df['Disasters'],
    mode='markers', 
    name='Verband tussen broeikasgasemissies en aantal natuurrampen',
    marker=dict(
        color=merged_all_df['Year'], 
        size=10, 
        colorscale='Viridis', 
        showscale=True
    )
))

# Update layout
fig.update_layout(
    title='Verband tussen broeikasgasemissies en aantal natuurrampen',
    xaxis_title='Emissies',
    yaxis_title='Rampen',
    template='plotly_white'
)

# Show figure
fig.show()

> Figuur 6: Deze grafiek toont de relatie tussen jaarlijkse broeikasgasemissies en het aantal natuurrampen van 1970 tot 2022.<br> Elk punt vertegenwoordigt een jaar, waarbij de kleurgradaties de tijdsprogressie aangeven.